[![img/pythonista.png)](img/pythonista.png)](https://www.pythonista.io)

# *Blueprints*.

Los *blueprints* son objetos similares a ```app```, los cuales son instanciados de la clase ```flask.Blueprint```.

```
<blueprint> = Blueprint("<nombre>", __name__)
```

Estos objetos permiten relacionar rutas y funciones de forma similar a ```app```, pero permiten su reutilización.\

La ventaja de los *blueprints* es que peuden ser reutilizados y funcionan de forma modular.

https://flask.palletsprojects.com/en/2.0.x/blueprints/

**Ejemplo:** 

* Dentro de la estructura del directorio [```src/proyecto-flaskabc_alumnos/```](src/proyecto-flask/abc_alumnos/) se encuentra el script [```blueprints.py```](/src/proyecto-flask/abc_alumnos/blueprints.py). En dicho scrpit se hace la implantación de un blueprint muy similar al código de la *API* que se ha utilizado a lo largo del curso.

* una diferencia notable es que se ha instanciado el objeto ```abc_alumno``` a partir de la clase ```flask.Blueprint```. 

```python 
'''Blueprints de la aplicación.'''
from flask import Blueprint, request, abort, jsonify
from models import db, Alumno
from functions import *
abc_alumno = Blueprint('abc_alumno', __name__)
```

* Se definen una serie de rutas y funciones de forma idéntica a como se hace con un objeto ```app```.

```python
@abc_alumno.route('/', methods=['GET'])
def muestra_alumnos():
    alumnos = Alumno.query.all()
    return jsonify([convierte_a_dict(alumno.cuenta, alumno, esquema_alumno) for alumno in alumnos])


@abc_alumno.route('/<cuenta>', methods=['GET', 'POST', 'DELETE', 'PUT'])
def ops_alumno(cuenta):
    if request.method == 'GET':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            return jsonify(convierte_a_dict(cuenta, alumno, esquema_alumno))
        else:
            abort(404)
    if request.method == 'DELETE':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            db.session.delete(alumno)
            db.session.commit()
            return jsonify(convierte_a_dict(cuenta, alumno, esquema_alumno))
        else:
            abort(404)      
    if request.method == 'POST':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            abort(403)
        else:
            respuesta = recurso_completo(cuenta, request.data)
            if respuesta:
                db.session.add(respuesta)
                db.session.commit()
                return jsonify(convierte_a_dict(cuenta, respuesta, esquema_alumno))
            else:
                abort(406)
    if request.method == 'PUT':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            respuesta = recurso_completo(cuenta, request.data)
            if respuesta:
                db.session.delete(alumno)
                db.session.add(respuesta)
                db.session.commit()
                return jsonify(convierte_a_dict(cuenta, respuesta, esquema_alumno))
            else:
                abort(406)
        else:
            
            abort(404)                 
```

## Registro de un *blueprint* en una aplicación.

Los *blueprints* pueden ser aprovechado por app mediante el método ```app.register_blueprint()```.

**Sintaxis:**

```
app.register_blueprint(<blueprint>, url_prefix='<ruta>')
```

**Ejemplo:**

* El script [```app.py```](/src/proyecto-flask/abc_alumnos/app.py) registra el blueprint ```abc_alumno``` para que atienda peticiones desde la ruta ```/api/```.

```python
'''Módulo principal de la aplicación.'''
from flask import Flask
from models import db
from blueprints import abc_alumno
app = Flask(__name__)
app.config.from_pyfile('settings.cfg')
db.init_app(app)
app.register_blueprint(abc_alumno, url_prefix='/api')

if __name__ == '__main__':
    app.run(host='0.0.0.0')
```

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>